In [ ]:
!pip install pytrends


In [ ]:
from pytrends import TrendReq

# Create a TrendReq object with the desired country and language
pytrends = TrendReq()
pytrends.build_payload(kw_list=['flooring'], cat=0, timeout=10, proxies=None)

# Get trending data for your keyword
data = pytrends.trending_searches(pn='united_states')

# Print the results
print(data)

In [ ]:
from pytrends.request import TrendReq
import pandas as pd
import time
import random
from requests.exceptions import RequestException

def exponential_backoff(attempt):
    return random.uniform(2, 4) ** attempt

def query_google_trends(kw_list, timeframe, max_retries=5):
    for attempt in range(max_retries):
        try:
            # Connect to Google Trends
            pytrends = TrendReq(hl='en-US', tz=360)

            # Build the payload
            pytrends.build_payload(kw_list, cat=0, timeframe=timeframe, geo='', gprop='')

            # Get interest over time
            interest_over_time_df = pytrends.interest_over_time()

            # Get related topics
            related_topics = pytrends.related_topics()

            # Get related queries
            related_queries = pytrends.related_queries()

            return interest_over_time_df, related_topics, related_queries

        except RequestException as e:
            if attempt < max_retries - 1:
                wait_time = exponential_backoff(attempt)
                print(f"Request failed. Retrying in {wait_time:.2f} seconds...")
                time.sleep(wait_time)
            else:
                print("Max retries reached. Unable to fetch data.")
                raise e

# Define the keyword(s) to search for
kw_list = ["Python programming"]

# Define the time frame (last 7 days)
timeframe = 'now 7-d'

try:
    interest_over_time_df, related_topics, related_queries = query_google_trends(kw_list, timeframe)

    # Print results
    print("Interest over time:")
    print(interest_over_time_df)

    print("\nRelated topics:")
    print(related_topics)

    print("\nRelated queries:")
    print(related_queries)

    # Save results to CSV files
    interest_over_time_df.to_csv('interest_over_time.csv')
    pd.DataFrame(related_topics[kw_list[0]]['top']).to_csv('related_topics.csv')
    pd.DataFrame(related_queries[kw_list[0]]['top']).to_csv('related_queries.csv')

    # Created/Modified files during execution:
    print("Created/Modified files:")
    for file_name in ["interest_over_time.csv", "related_topics.csv", "related_queries.csv"]:
        print(file_name)

except Exception as e:
    print(f"An error occurred: {str(e)}")

In [1]:
from pytrends.request import TrendReq

pytrends=TrendReq(hl='en-US', tz=360)

keywords=['flooring']
pytrends.build_payload(keywords, timeframe='today 5-y')
interest_over_time_df = pytrends.interest_over_time()

TooManyRequestsError: The request failed: Google returned a response with code 429